In [12]:
import fitz  # PyMuPDF
import csv
import re
import numpy as np 
import pandas as pd

# 1. 讀取 PDF 中的文字
pdf_file = "files/JK37415119.pdf"
doc = fitz.open(pdf_file)

invoice_data = {
    "發票號碼": "",
    "發票日期": "",
    "銷售額合計": 0,
    "總計": 0
}

for page_num in range(len(doc)):
    page = doc[page_num]
    text = page.get_text()

    # 2. 提取發票號碼和發票日期
    if not invoice_data["發票號碼"]:
        match_invoice = re.search(r'發票號碼:(\w+)', text)
        if match_invoice:
            invoice_data["發票號碼"] = match_invoice.group(1)
    
    if not invoice_data["發票日期"]:
        match_date = re.search(r'(\d{4}-\d{2}-\d{2})', text)
        if match_date:
            invoice_data["發票日期"] = match_date.group(1)


    # 4. 提取銷售額合計與總計
    match_sales_total = re.search(r'銷售額合計\s+([\d,]+)', text)
    match_grand_total = re.search(r'總計\s+([\d,]+)', text)

    if match_sales_total:
        invoice_data["銷售額合計"] = match_sales_total.group(1).replace(",", "")
    if match_grand_total:
        invoice_data["總計"] = match_grand_total.group(1).replace(",", "")

        invoice_data["銷售額合計"] = match_sales_total.group(1).replace(",", "")



# 5. 將資料寫入 CSV 檔案
output_file = "invoice_data_with_date.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 寫入標題
    writer.writerow(["發票號碼", "發票日期", "銷售額合計", "總計"])
    
    
    # 寫入品項資料，每行加上日期、銷售額合計與總計
   

    writer.writerow([
            invoice_data["發票號碼"],
            invoice_data["發票日期"],
            invoice_data["銷售額合計"],
            invoice_data["總計"]
        ])

    print(f"發票資料已存成 CSV 檔案：{output_file}")

    idata= np.array(invoice_data)
    df = pd.DataFrame(data=idata, columns=["發票號碼", "發票日期", "銷售額合計", "總計"])


發票資料已存成 CSV 檔案：invoice_data_with_date.csv


ValueError: Must pass 2-d input. shape=()

In [13]:
import pandas as pd
df = pd.read_csv(output_file)
df

,發票號碼,發票日期,銷售額合計,總計
0,JK37415119,2025-02-03,2895,3040
